In [ ]:
from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU, recall_func
from sklearn.metrics import roc_auc_score
from MF_synthetic import MF, MF_IPS, MF_Multi_IPS, MF_ESCM2_IPS, MF_DT_IPS
from MF_synthetic import MF_DR, MF_DR_JL, MF_Multi_DR, MF_ESCM2_DR, MF_DT_DR
import torch
import pickle
import numpy as np

mse_func = lambda x,y: np.mean((x-y)**2)
mae_func = lambda x,y: np.mean(np.abs(x-y))

num_user = 1682
num_item = 943

def generate_total_sample(num_user, num_item):
    sample = []
    for i in range(num_user):
        sample.extend([[i,j] for j in range(num_item)])
    return np.array(sample)

file = open("data/synthetic_data_100k", "rb")
ground_truth = pickle.load(file)
propensity = pickle.load(file)
file.close()

alpha = 0.75
propensity = propensity ** alpha

mse_oursdr = []
mae_oursdr = []
ndcg50_oursdr = []

mse_oursips = []
mae_oursips = []
ndcg50_oursips = []

mse_ips = []
mae_ips = []
ndcg50_ips = []

mse_mf = []
mae_mf = []
ndcg50_mf = []

mse_dr = []
mae_dr = []
ndcg50_dr = []

mse_mips = []
mae_mips = []
ndcg50_mips = []

mse_mdr = []
mae_mdr = []
ndcg50_mdr = []

mse_eips = []
mae_eips = []
ndcg50_eips = []

mse_edr = []
mae_edr = []
ndcg50_edr = []
for i in range(50):
    
    observation = np.random.binomial(1, propensity)

    y_train = ground_truth[observation == 1]
    
    o = np.where(observation == 1)[0]
    x_train = np.c_[np.array(o) % num_user, np.array(o) // num_user]
    
    y_test = ground_truth.reshape([num_user, num_item])
    x_test = generate_total_sample(num_user, num_item)
    
    mf_udr = MF_DR_jieou_new(num_user, num_item, embedding_k = 8, embedding_k1 = 8, batch_size = 4096)
    mf_udr.cuda()

    mf_udr.fit(x_train, y_train, alpha = 0.5, theta = 1,
        lr=0.01,
        gamma = 0.05,
        G = 1,
        num_epoch = 300,
        lamb1 = 1e-5,
        lamb2 = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)
    
    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])
    
    mse_oursdr.append(mse)
    mae_oursdr.append(mae)
    ndcg50_oursdr.append(np.mean(ndcg_res["ndcg_50"]))  

    mf_udr = MF_IPS_jieou_new(num_user, num_item, embedding_k = 8, embedding_k1 = 8, batch_size = 4096)
    mf_udr.cuda()

    mf_udr.fit(x_train, y_train, alpha = 0.5,
        lr=0.01,
        gamma = 0.05,
        G = 1,
        num_epoch = 300,
        lamb1 = 1e-5,
        lamb2 = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)

    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])
    

    mse_oursips.append(mse)
    mae_oursips.append(mae)
    ndcg50_oursips.append(np.mean(ndcg_res["ndcg_50"]))  

    
    mf_udr = MF_IPS(num_user, num_item, embedding_k = 8, batch_size = 4096, batch_size_prop = 40960)
    mf_udr.cuda()
    mf_udr._compute_IPS(x_train, lr = 0.01, lamb = 1e-5)
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        gamma = 0.05,
        lamb = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)
    
    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])


    mse_ips.append(mse)
    mae_ips.append(mae)
    ndcg50_ips.append(np.mean(ndcg_res["ndcg_50"]))  

    
    mf_udr = MF(num_user, num_item, embedding_k = 8, batch_size = 4096)
    mf_udr.cuda()
    # mf_udr._compute_IPS(x_train, lr = 0.01, lamb = 5*1e-4)
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        lamb = 1e-4,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)    
    
    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])
    
    mse_mf.append(mse)
    mae_mf.append(mae)
    ndcg50_mf.append(np.mean(ndcg_res["ndcg_50"]))  
  
    mf_udr = MF_DR_JL(num_user, num_item, embedding_k = 8, batch_size = 4096, batch_size_prop = 40960)
    mf_udr.cuda()
    mf_udr._compute_IPS(x_train, lr = 0.01, lamb = 1e-5)
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        gamma = 0.05,
        lamb = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)

    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])
    
    mse_dr.append(mse)
    mae_dr.append(mae)
    ndcg50_dr.append(np.mean(ndcg_res["ndcg_50"]))  

    mf_udr = MF_Multi_IPS(num_user, num_item, embedding_k = 8, batch_size = 4096, batch_size_prop = 40960)
    mf_udr.cuda()
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        gamma = 0.05,
        lamb = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)

    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])

    mse_mips.append(mse)
    mae_mips.append(mae)
    ndcg50_mips.append(np.mean(ndcg_res["ndcg_50"]))  
  
    mf_udr = MF_Multi_DR(num_user, num_item, embedding_k = 8, batch_size = 4096, batch_size_prop = 40960)
    mf_udr.cuda()
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        gamma = 0.05,
        lamb = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)
    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])


    mse_mdr.append(mse)
    mae_mdr.append(mae)
    ndcg50_mdr.append(np.mean(ndcg_res["ndcg_50"]))     
    
    mf_udr = MF_ESCM2_IPS(num_user, num_item, embedding_k = 8, batch_size = 4096)
    mf_udr.cuda()
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        gamma = 0.05,
        lamb = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)

    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])

    mse_eips.append(mse)
    mae_eips.append(mae)
    ndcg50_eips.append(np.mean(ndcg_res["ndcg_50"]))       
    
    mf_udr = MF_ESCM2(num_user, num_item, embedding_k = 8, batch_size = 4096)
    mf_udr.cuda()
    mf_udr.fit(x_train, y_train,
        lr=0.01,
        gamma = 0.05,
        lamb = 1e-5,
        tol=1e-5)
    test_pred = mf_udr.predict(x_test)
    mse = mse_func(ground_truth, test_pred)
    mae = mae_func(ground_truth, test_pred)  
    ndcg_res = ndcg_func(mf_udr, x_test, ground_truth, top_k_list = [50])

    mse_edr.append(mse)
    mae_edr.append(mae)
    ndcg50_edr.append(np.mean(ndcg_res["ndcg_50"]))  